***
# Merge data
***

In [1]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

Le dossier « Dataframe » est existant
Le dossier « Output » est existant
Le dossier « Data » est existant

c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Data\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Dataframe\
c:\Users\Charles_tour\Documents\GitHub\car_sales_forcast\Output\


In [2]:
import concurrent.futures
import numpy as np
import pandas as pd
import re
import time

pd.set_option('display.max_columns', None)

In [3]:
# Import data
scrap_data = pd.read_hdf(path_df + 'scrap_data.h5')

vente_2017 = pd.read_hdf(path_df + 'vente_data_2017.h5')
vente_2018 = pd.read_hdf(path_df + 'vente_data_2018.h5')

In [4]:
# Scrap_data key column
key_col = ['brand', 'model', 'year']

scrap_data['merge_key'] = scrap_data[key_col].apply(lambda x: x.str.upper()).apply(lambda x: '_'.join(x), axis=1)
scrap_data.head(3)

,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max,merge_key
0,https://www.guideautoweb.com/constructeurs/acu...,2023,integra,acura,34350.0,42550.0,6.3,8.9,ACURA_INTEGRA_2023
1,https://www.guideautoweb.com/constructeurs/acu...,2023,mdx,acura,59300.0,84400.0,9.4,13.8,ACURA_MDX_2023
2,https://www.guideautoweb.com/constructeurs/acu...,2022,mdx,acura,57900.0,83000.0,9.4,13.8,ACURA_MDX_2022


In [5]:
# Creer un dictionnaire contenant df pour chaque annéée
years = range(2017,2019)
dfs = [vente_2017, vente_2018]

df_dict = {}
for df, year in zip(dfs, years):
    df_dict[year] = df

In [6]:
# Colonne key pour df vente
key_col = ['MARQ_VEH', 'MODEL_VEH', 'ANNEE_MOD']

for year in years:
    df_dict[year]['ANNEE_MOD'] = df_dict[year]['ANNEE_MOD'].astype(int)
    df_dict[year]['merge_key'] = df_dict[year][key_col].astype(str).apply(lambda x: '.*'.join(x), axis=1)

df_dict[2017].head(2)

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,NB_ESIEU_MAX,COUL_ORIG,TYP_CARBU,TYP_DOSS_PERS,PHYS_SEX,PHYS_AGE,REG_ADM,MRC,CG_FIXE,merge_key
0,2017,2017_0000000016,PAU,AU,FORD,ESCAP,2017,1662.0,4.0,2001.0,NaN,ROU,E,P,F,76.0,Centre-du-Québec (17),Drummond (49 ),49030.0,FORD.*ESCAP.*2017
1,2017,2017_0000000022,PAU,AU,CHEVR,VOLT,2017,1578.0,4.0,1509.0,NaN,NOI,W,P,M,33.0,Saguenay–Lac-Saint-Jean (02),Saguenay (941),94068.0,CHEVR.*VOLT.*2017


In [16]:
# Associer une clef à chaque immatriculation des ventes
start_time = time.time()

# def find_best_match(df_to_match):
#     matches = [x for x in scrap_data['merge_key'].values if re.findall(df_to_match, x)]
#     return min(matches, key=len) if matches else None

def find_best_match(df_to_match):
    matche = [x for x in scrap_data['merge_key'].values if re.search(df_to_match, x)]
    return matche[0] if matche else None

def time_duration(start_time):
    end_time = time.time()
    duration = (end_time - start_time)/60
    print(f'{duration} min')

with concurrent.futures.ThreadPoolExecutor() as executor:
    for year in years:
        df_dict[year]['match'] = list(executor.map(find_best_match, df_dict[year]['merge_key']))
        time_duration(start_time)
# 15m

862.575501203537
1695.5375926494598


In [18]:
df_dict[2017].head(2)

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,...,COUL_ORIG,TYP_CARBU,TYP_DOSS_PERS,PHYS_SEX,PHYS_AGE,REG_ADM,MRC,CG_FIXE,merge_key,match
0,2017,2017_0000000016,PAU,AU,FORD,ESCAP,2017,1662.0,4.0,2001.0,...,ROU,E,P,F,76.0,Centre-du-Québec (17),Drummond (49 ),49030.0,FORD.*ESCAP.*2017,FORD_ESCAPE_2017
1,2017,2017_0000000022,PAU,AU,CHEVR,VOLT,2017,1578.0,4.0,1509.0,...,NOI,W,P,M,33.0,Saguenay–Lac-Saint-Jean (02),Saguenay (941),94068.0,CHEVR.*VOLT.*2017,CHEVROLET_VOLT_2017


In [31]:
# Nombre de match
print('STATISTIQUE DES MATCHES')

for year in years:
    nb_observation = len(df_dict[year])
    nb_match = len(df_dict[year].loc[df_dict[year]['match']!=None])
    non_match = nb_observation - nb_match
    
    print('-' * 40)
    print(f'Année : {year}')
    print(f'Nombre de vente : {nb_observation}')
    print(f'Nombre de match : {nb_match}')
    print(f'Nombre d\'observation non matchée : {non_match}')

STATISTIQUE DES MATCHES
----------------------------------------
Année : 2017
Nombre de vente : 496045
Nombre de match : 496045
Nombre d'observation non matchée : 0
----------------------------------------
Année : 2018
Nombre de vente : 477643
Nombre de match : 477643
Nombre d'observation non matchée : 0


In [32]:
# Merge

merge_data = {}
for year in years:
    merge_data[year] = df_dict[year].merge(scrap_data, how='left', left_on='match', right_on='merge_key', indicator=True)
merge_data[2017].head()

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,...,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max,merge_key_y,_merge
0,2017,2017_0000000016,PAU,AU,FORD,ESCAP,2017,1662.0,4.0,2001.0,...,https://www.guideautoweb.com/constructeurs/for...,2017,escape,ford,25099.0,35999.0,7.8,11.5,FORD_ESCAPE_2017,both
1,2017,2017_0000000022,PAU,AU,CHEVR,VOLT,2017,1578.0,4.0,1509.0,...,https://www.guideautoweb.com/constructeurs/che...,2017,volt,chevrolet,38790.0,42890.0,5.5,5.6,CHEVROLET_VOLT_2017,both
2,2017,2017_0000000041,PAU,AU,NISSA,VERSA,2017,1113.0,4.0,1607.0,...,https://www.guideautoweb.com/constructeurs/nis...,2017,versa-note,nissan,14498.0,19748.0,6.2,8.6,NISSAN_VERSA-NOTE_2017,both
3,2017,2017_0000000044,PAU,AU,TOYOT,YARIS,2018,1050.0,4.0,1492.0,...,https://www.guideautoweb.com/constructeurs/toy...,2018,yaris,toyota,15490.0,20530.0,5.8,7.9,TOYOTA_YARIS_2018,both
4,2017,2017_0000000071,PAU,AU,NISSA,PATHF,2017,2018.0,6.0,3509.0,...,https://www.guideautoweb.com/constructeurs/nis...,2017,pathfinder,nissan,32598.0,48598.0,8.5,12.4,NISSAN_PATHFINDER_2017,both


In [33]:
merge_data[2018].head()

,AN,NOSEQ_VEH,CLAS,TYP_VEH_CATEG_USA,MARQ_VEH,MODEL_VEH,ANNEE_MOD,MASSE_NETTE,NB_CYL,CYL_VEH,...,link_model_an,year,model,brand,prix_min,prix_max,cons_min,cons_max,merge_key_y,_merge
0,2018,2018_0000000011,PAU,AU,BMW,X3,2018,1885.0,4.0,2001.0,...,https://www.guideautoweb.com/constructeurs/bmw...,2018,x3,bmw,48000.0,61500.0,8.0,11.7,BMW_X3_2018,both
1,2018,2018_0000000020,PAU,AU,FORD,F150,2018,2150.0,6.0,2706.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,2018,2018_0000000023,PAU,AU,NISSA,KICKS,2019,1430.0,4.0,1607.0,...,https://www.guideautoweb.com/constructeurs/nis...,2019,kicks,nissan,18298.0,23398.0,6.6,7.7,NISSAN_KICKS_2019,both
3,2018,2018_0000000030,PAU,AU,ACURA,RDX,2018,1703.0,6.0,3509.0,...,https://www.guideautoweb.com/constructeurs/acu...,2018,rdx,acura,42390.0,47390.0,8.7,12.4,ACURA_RDX_2018,both
4,2018,2018_0000000045,CAU,AU,CHEVR,MALIB,2018,1401.0,4.0,1509.0,...,https://www.guideautoweb.com/constructeurs/che...,2018,malibu,chevrolet,24695.0,32495.0,4.8,10.5,CHEVROLET_MALIBU_2018,both


In [34]:
# Clean and save
for year in years : 
    merge_data[year] = merge_data[year].drop(columns=['merge_key_x', 'match', 'merge_key_y', '_merge'])

    merge_data[year].to_hdf(path_df + 'merge_data_{year}.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_10280\958229151.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['NOSEQ_VEH', 'CLAS', 'TYP_VEH_CATEG_USA', 'MARQ_VEH', 'MODEL_VEH',
       'COUL_ORIG', 'TYP_CARBU', 'TYP_DOSS_PERS', 'PHYS_SEX', 'REG_ADM', 'MRC',
       'link_model_an', 'year', 'model', 'brand'],
      dtype='object')]

  merge_data[year].to_hdf(path_df + 'merge_data_{year}.h5', key='s')
C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_10280\958229151.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['NOSEQ_VEH', 'CLAS', 'TYP_VEH_CATEG_USA', 'MARQ_VEH', 'MODEL_VEH',
       'COUL_ORIG', 'TYP_CARBU', 'TYP_DOSS_PERS', 'PHYS_SEX', 'REG_ADM', 'MRC',
       'link_model_an', 'year', 'model', 'brand'],
 